## Train with pre-trained ResNet-18 Model

Fine-tune a pre-trained ResNet-18 model and achieve at least 70% test accuracy.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import h5py
import datetime

import torch
import torch.nn as nn
import torch.utils.data as Data

In [2]:
from torch.utils import model_zoo

In [3]:
import torchvision
from torchvision import transforms

In [4]:
# Data Augmentation
ds_trans = transforms.Compose([transforms.Resize(224),
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomVerticalFlip(),
                              transforms.ColorJitter(
                                brightness=0.1*torch.randn(1),
                                contrast=0.1*torch.randn(1),
                                saturation=0.1*torch.randn(1),
                                hue=0.1*torch.randn(1)),
                             transforms.ToTensor()])

In [5]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
# Load data
BatchSize = 100
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True,
                                        transform=ds_trans)
train_loader = Data.DataLoader(trainset, batch_size=BatchSize, shuffle=True)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True,
                                       transform=ds_trans)
test_loader = torch.utils.data.DataLoader(testset, batch_size=BatchSize, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Define pre-trained model ResNet18
def resnet18(pretrained=True):
    model = torchvision.models.resnet.ResNet(torchvision.models.resnet.BasicBlock,[2,2,2,2])
    if pretrained:
        model.load_state_dict(model_zoo.load_url('https://download.pytorch.org/models/resnet18-5c106cde.pth',model_dir="./"))
    return model

In [8]:
model = resnet18()

In [9]:
import copy
model2 = copy.deepcopy(model)

In [10]:
# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)

In [11]:
for i,(images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device).long()
            
    # Forward
    print(images.shape)
    outputs = model(images)
    
    if i>=5:
        break

torch.Size([100, 3, 224, 224])


KeyboardInterrupt: 

In [23]:
# Model training process
start_time = datetime.datetime.now()
num_epochs = 2
num_steps = len(train_loader)

for epoch in range(num_epochs):
    total = 0
    correct = 0 # Count accuracy in each epoch
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device).long()
            
        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (labels == predicted).sum().item()
            
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        if (i+1)%100 == 0:
            print("Epoch[{}/{}], Step[{}/{}], Loss {:4f}, Accuracy {:4f}%".format(
            epoch+1, num_epochs, i+1, num_steps, loss.item(), correct/total*100))
            now_time = datetime.datetime.now()
            print("Total cost time:{}".format(now_time-start_time))

RuntimeError: Given input size: (512x1x1). Calculated output size: (512x-5x-5). Output size is too small at c:\programdata\miniconda3\conda-bld\pytorch_1532505617613\work\aten\src\thnn\generic/SpatialAveragePooling.c:64